# **Código nacional**
Esto es una prueba. Esto será el código para tratar todo nacional.

## Instalación de paquetes

In [1]:
!pip install geopandas
!pip install dbfread

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF

## Paso dos - Información censal

In [3]:
#DATOS PARA PASO DOS

#RUTAS ENTRADA
MZNA_2020_01_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_01CSV20.csv"
MZNA_2020_02_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_02CSV20.csv"
MZNA_2020_03_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_03CSV20.csv"
MZNA_2020_04_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_04CSV20.csv"
MZNA_2020_05_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_05CSV20.csv"
MZNA_2020_06_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_06CSV20.csv"
MZNA_2020_07_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_07CSV20.csv"
MZNA_2020_08_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_08CSV20.csv"
MZNA_2020_09_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_09CSV20.csv"
MZNA_2020_10_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_10CSV20.csv"
MZNA_2020_11_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_11CSV20.csv"
MZNA_2020_12_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_12CSV20.csv"
MZNA_2020_13_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_13CSV20.csv"
MZNA_2020_14_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_14CSV20.csv"
MZNA_2020_15_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_15CSV20.csv"
MZNA_2020_16_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_16CSV20.csv"
MZNA_2020_17_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_17CSV20.csv"
MZNA_2020_18_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_18CSV20.csv"
MZNA_2020_19_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_19CSV20.csv"
MZNA_2020_20_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_20CSV20.csv"
MZNA_2020_21_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_21CSV20.csv"
MZNA_2020_22_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_22CSV20.csv"
MZNA_2020_23_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_23CSV20.csv"
MZNA_2020_24_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_24CSV20.csv"
MZNA_2020_25_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_25CSV20.csv"
MZNA_2020_26_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_26CSV20.csv"
MZNA_2020_27_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_27CSV20.csv"
MZNA_2020_28_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_28CSV20.csv"
MZNA_2020_29_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_29CSV20.csv"
MZNA_2020_30_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_30CSV20.csv"
MZNA_2020_31_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_31CSV20.csv"
MZNA_2020_32_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_32CSV20.csv"

MZNA_2010_01_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_01DBF10.dbf"
MZNA_2010_02_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_02DBF10.dbf"
MZNA_2010_03_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_03DBF10.dbf"
MZNA_2010_04_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_04DBF10.dbf"
MZNA_2010_05_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_05DBF10.dbf"
MZNA_2010_06_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_06DBF10.dbf"
MZNA_2010_07_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_07DBF10.dbf"
MZNA_2010_08_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_08DBF10.dbf"
MZNA_2010_09_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_09DBF10.dbf"
MZNA_2010_10_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_10DBF10.dbf"
MZNA_2010_11_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_11DBF10.dbf"
MZNA_2010_12_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_12DBF10.dbf"
MZNA_2010_13_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_13DBF10.dbf"
MZNA_2010_14_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_14DBF10.dbf"
MZNA_2010_15_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_15DBF10.dbf"
MZNA_2010_16_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_16DBF10.dbf"
MZNA_2010_17_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_17DBF10.dbf"
MZNA_2010_18_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_18DBF10.dbf"
MZNA_2010_19_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_19DBF10.dbf"
MZNA_2010_20_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_20DBF10.dbf"
MZNA_2010_21_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_21DBF10.dbf"
MZNA_2010_22_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_22DBF10.dbf"
MZNA_2010_23_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_23DBF10.dbf"
MZNA_2010_24_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_24DBF10.dbf"
MZNA_2010_25_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_25DBF10.dbf"
MZNA_2010_26_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_26DBF10.dbf"
MZNA_2010_27_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_27DBF10.dbf"
MZNA_2010_28_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_28DBF10.dbf"
MZNA_2010_29_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_29DBF10.dbf"
MZNA_2010_30_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_30DBF10.dbf"
MZNA_2010_31_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_31DBF10.dbf"
MZNA_2010_32_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_32DBF10.dbf"


ITER_2020_ruta = r"C:\Users\Alejandro\Downloads\DATA\2020\ITER\ITER_NALCSV20.csv"
ITER_2010_ruta = r"C:\Users\Alejandro\Downloads\DATA\2010\ITER\ITER_NALDBF10.dbf"


#RUTAS SALIDA
MZNA_2020_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\MZNA_2020_CYA.csv"
ITER_2020_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\MZNA_2010_CYA.csv"
MZNA_2010_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\ITER_2020_CYA.csv"
ITER_2010_salida = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\ITER_2010_CYA.csv"


In [5]:
import geopandas as gpd

# Lista para almacenar los DataFrames
dfs = []

# Iterar sobre los archivos y leer cada uno
for i in range(1, 33):
    filename = f"MZNA_2010_{i:02d}_ruta"
    # Leer el archivo y agregarlo al DataFrame
    df = gpd.read_file(eval(filename))
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
combined_df = pd.concat(dfs, ignore_index=True)

# Verificar el DataFrame combinado
print(combined_df.head())


KeyboardInterrupt: 

In [ ]:
#PASO DOS - Preparación de información censal




# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    table = DBF(file_path)
    return pd.DataFrame(iter(table))

# Cargar archivos en un DataFrame
MZNA_2020_df = pd.read_csv(MZNA_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'AGEB': str, 'MZA': str})
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'LATITUD': str})

MZNA_2010_df = load_dbf_to_dataframe(MZNA_2010_ruta)
ITER_2010_df = load_dbf_to_dataframe(ITER_2010_ruta)

# 2.2 Homologación de ID urbana - CVEGEO
def homologate_urban_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}{row['AGEB']}{row['MZA']}"

# 3.2 Homologación de ID rural - CVEGEO
def homologate_rural_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}"

MZNA_2020_df['CVEGEO'] = MZNA_2020_df.apply(homologate_urban_id, axis=1)
MZNA_2010_df['CVEGEO'] = MZNA_2010_df.apply(homologate_urban_id, axis=1)

ITER_2020_df['CVEGEO'] = ITER_2020_df.apply(homologate_rural_id, axis=1)
ITER_2010_df['CVEGEO'] = ITER_2010_df.apply(homologate_rural_id, axis=1)

#reemplazar * con 0
def replace(dataframe):
    return dataframe.replace('*', '0')

MZNA_2020_df_rep = replace(MZNA_2020_df)
MZNA_2010_df_rep = replace(MZNA_2010_df)
ITER_2020_df_rep = replace(ITER_2020_df)
ITER_2010_df_rep = replace(ITER_2010_df)

# Función para filtrar información por municipios correspondientes
def filtrar_municipios(dataframe, municipios):
    return dataframe[dataframe['MUN'].isin(municipios)]

# aplicar filtrado por municipios
MZNA_2020_df_filter = filtrar_municipios(MZNA_2020_df_rep, municipios)
MZNA_2010_df_filter = filtrar_municipios(MZNA_2010_df_rep, municipios)

#Filtrar ITER
def filtrar_municipios_iter(dataframe, municipios, entidad):
    return dataframe[(dataframe['MUN'].isin(municipios)) & (dataframe['ENTIDAD'].isin(entidad))]

ITER_2020_df_filter = filtrar_municipios_iter(ITER_2020_df_rep, municipios, entidad)
ITER_2010_df_filter = filtrar_municipios_iter(ITER_2010_df_rep, municipios, entidad)

# 2.3 Guardado de información en CSV
def save_to_csv(dataframe, filename):
    dataframe.to_csv(filename, index=False)

save_to_csv(MZNA_2020_df_filter, MZNA_2020_salida)
save_to_csv(MZNA_2010_df_filter, ITER_2020_salida)
save_to_csv(ITER_2020_df_filter, MZNA_2010_salida)
save_to_csv(ITER_2010_df_filter, ITER_2010_salida)

In [ ]:
import pandas as pd
from simpledbf import Dbf5
import os

# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    dbf = Dbf5(file_path)
    return dbf.to_dataframe()

# Función para aplicar homologación y reemplazos necesarios
def process_dataframe(dataframe, urbano=True):
    # Homologación de ID
    if urbano:
        dataframe['CVEGEO'] = dataframe.apply(lambda row: f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}{row['AGEB']}{row['MZA']}", axis=1)
    else:
        dataframe['CVEGEO'] = dataframe.apply(lambda row: f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}", axis=1)
    # Reemplazo de * por 0
    dataframe.replace('*', '0', inplace=True)
    return dataframe

# Directorio de datos
data_dir = 'ruta/a/los/datos'

# Años y tipos de ubicación para iterar
years = ['2010', '2020']
locations = ['MZNA', 'ITER']

# Iterar sobre los años, tipos de ubicación y estados
for year in years:
    for location in locations:
        for state_folder in os.listdir(os.path.join(data_dir, year)):
            state_path = os.path.join(data_dir, year, state_folder)
            for file in os.listdir(state_path):
                if file.startswith(location):
                    file_path = os.path.join(state_path, file)
                    if file.endswith('.dbf'):
                        df = load_dbf_to_dataframe(file_path)
                    else:
                        df = pd.read_csv(file_path, dtype=str)
                    df_processed = process_dataframe(df, urbano=(location == 'MZNA'))
                    # Guardar el DataFrame procesado
                    output_folder = os.path.join('salida', year, state_folder)
                    os.makedirs(output_folder, exist_ok=True)
                    output_path = os.path.join(output_folder, f"{location}_{year}_{state_folder}.csv")
                    df_processed.to_csv(output_path, index=False)

print("Procesamiento completado.")


## Pasos tres a seis: Información geo-estadística

In [ ]:
#DATOS PARA PASO TRES A SEIS

#SUBIR SHP 32 ESTADOS

#rutas shp base
SHP_MZN_01_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\01m.shp"
SHP_LOC_01_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\01l.shp"

SHP_MZN_02_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\02m.shp"
SHP_LOC_02_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\02l.shp"

SHP_MZN_03_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\03m.shp"
SHP_LOC_03_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\03l.shp"

SHP_MZN_04_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\04m.shp"
SHP_LOC_04_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\04l.shp"

SHP_MZN_05_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\05m.shp"
SHP_LOC_05_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\05l.shp"

SHP_MZN_06_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\06m.shp"
SHP_LOC_06_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\06l.shp"

SHP_MZN_07_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\07m.shp"
SHP_LOC_07_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\07l.shp"

SHP_MZN_08_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\08m.shp"
SHP_LOC_08_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\08l.shp"

SHP_MZN_09_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\09m.shp"
SHP_LOC_09_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\09l.shp"

SHP_MZN_10_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\10m.shp"
SHP_LOC_10_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\10l.shp"

SHP_MZN_11_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\11m.shp"
SHP_LOC_11_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\11l.shp"

SHP_MZN_12_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\12m.shp"
SHP_LOC_12_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\12l.shp"

SHP_MZN_13_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\13m.shp"
SHP_LOC_13_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\13l.shp"

SHP_MZN_14_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\14m.shp"
SHP_LOC_14_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\14l.shp"

SHP_MZN_15_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\15m.shp"
SHP_LOC_15_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\15l.shp"

SHP_MZN_16_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\16m.shp"
SHP_LOC_16_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\16l.shp"

SHP_MZN_17_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\17m.shp"
SHP_LOC_17_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\17l.shp"

SHP_MZN_18_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\18m.shp"
SHP_LOC_18_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\18l.shp"

SHP_MZN_19_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\19m.shp"
SHP_LOC_19_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\19l.shp"

SHP_MZN_20_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\20m.shp"
SHP_LOC_20_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\20l.shp"

SHP_MZN_21_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\21m.shp"
SHP_LOC_21_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\21l.shp"

SHP_MZN_22_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\22m.shp"
SHP_LOC_22_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\22l.shp"

SHP_MZN_23_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\23m.shp"
SHP_LOC_23_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\23l.shp"

SHP_MZN_24_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\24m.shp"
SHP_LOC_24_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\24l.shp"

SHP_MZN_25_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\25m.shp"
SHP_LOC_25_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\25l.shp"

SHP_MZN_26_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\26m.shp"
SHP_LOC_26_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\26l.shp"

SHP_MZN_27_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\27m.shp"
SHP_LOC_27_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\27l.shp"

SHP_MZN_28_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\28m.shp"
SHP_LOC_28_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\28l.shp"

SHP_MZN_29_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\29m.shp"
SHP_LOC_29_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\29l.shp"

SHP_MZN_30_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\30m.shp"
SHP_LOC_30_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\30l.shp"

SHP_MZN_31_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\31m.shp"
SHP_LOC_31_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\31l.shp"

SHP_MZN_32_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\32m.shp"
SHP_LOC_32_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\32l.shp"


#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\Mapa_Tipologías_CYA"


In [ ]:
#PASO TRES - Unión de información estadística y cartografía

#Carga de capas de manzanas y localidades a gdf
SHP_MZN_gdf = gpd.read_file(SHP_MZN_ruta)
SHP_LOC_gdf = gpd.read_file(SHP_LOC_ruta)

#Filtrar ITER
def filtrar_municipios_gpkg(dataframe, municipios, entidad):
    return dataframe[(dataframe['CVE_MUN'].isin(municipios)) & (dataframe['CVE_ENT'].isin(entidad))]

SHP_MZN_gdf_filter = filtrar_municipios_gpkg(SHP_MZN_gdf, municipios, entidad)
SHP_LOC_gdf_filter = filtrar_municipios_gpkg(SHP_LOC_gdf, municipios, entidad)

# CREAR GPKG BASE DESDE gdf DE MANZANAS
SHP_MZN_gdf_filter.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_gpkg, layer='Mapa_Tipologias')

# Selección de ITER rural
SHP_LOC_gdf_filter_RURAL = SHP_LOC_gdf_filter[SHP_LOC_gdf_filter['AMBITO'] == 'Rural']

# Concatenar ITER rural a la capa existente
gdf_concatenated = gpd.GeoDataFrame(pd.concat([MAPA_TIPO_gpkg_gdf, SHP_LOC_gdf_filter_RURAL], ignore_index=True))

# Guardar la capa concatenada en el GeoPackage base
gdf_concatenated.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG', overwrite=True)

# Re-leer la capa del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')


###COMBINAR 32 ESTADOS EN 1

##FILTRAR MUNICIPIOS --> DOCUMENTO / Lista con metrópolis


# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(MZNA_2020_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2020', '_2020_MZNA'))
MAPA_TIPO_gdf2 = MAPA_TIPO_gdf1.merge(ITER_2020_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2020', '_2020_ITER'))
MAPA_TIPO_gdf3 = MAPA_TIPO_gdf2.merge(MZNA_2010_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2010', '_2010_MZNA'))
MAPA_TIPO_gdf4 = MAPA_TIPO_gdf3.merge(ITER_2010_df[['CVEGEO', 'POBTOT', 'TVIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left', suffixes=('_2010', '_2010_ITER'))

#limpiar  *
MAPA_TIPO_gdf4_A = replace(MAPA_TIPO_gdf4)

#reemplazar N/D
MAPA_TIPO_gdf4_B = MAPA_TIPO_gdf4_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_B.fillna(0)

# Crear campos POPTOT2010, POPTOT2020, VIVPAR2010 y VIVPAR2020 y llenado respectivamente
MAPA_TIPO_gdf4_C['POPTOT2010'] = MAPA_TIPO_gdf4_B['POBTOT_2010'].astype(int) + MAPA_TIPO_gdf4_B['POBTOT_2010_ITER'].astype(int)
MAPA_TIPO_gdf4_C['POPTOT2020'] = MAPA_TIPO_gdf4_B['POBTOT_2020'].astype(int) + MAPA_TIPO_gdf4_B['POBTOT_2020_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR2010'] = MAPA_TIPO_gdf4_B['TVIVPAR_2010'].astype(int) + MAPA_TIPO_gdf4_B['TVIVPAR_2010_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR2020'] = MAPA_TIPO_gdf4_B['TVIVPAR_2020'].astype(int) +  MAPA_TIPO_gdf4_B['TVIVPAR_2020_ITER'].astype(int)
MAPA_TIPO_gdf4_C['VIVPARDES2020'] = MAPA_TIPO_gdf4_B['VIVPAR_DES_2020'].astype(int) + MAPA_TIPO_gdf4_B['VIVPAR_DES_2020_ITER'].astype(int)

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver="GPKG")

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Creación de campos y llenado de información
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVPAR2020'] - MAPA_TIPO_gdf['VIVPAR2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVPAR2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.select(
    [MAPA_TIPO_gdf['RELVIV'] < -0.075, (MAPA_TIPO_gdf['RELVIV'] >= -0.075) & (MAPA_TIPO_gdf['RELVIV'] <= 0.075), MAPA_TIPO_gdf['RELVIV'] > 0.075],
    [1, 2, 3]
)
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
MAPA_TIPO_gdf['TIPOLOGIA'] = np.where((MAPA_TIPO_gdf['POPTOT2020'] == 0) & (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVPAR2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3'), 'B3\'', MAPA_TIPO_gdf['CVE'])
MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
MAPA_TIPO_gdf['FILTRADO'] = MAPA_TIPO_gdf['VIVPAR2020'] / MAPA_TIPO_gdf['area_ha']**2

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver="GPKG")

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Paso 6: Filtrado de información
MAPA_TIPO_gdf_FILTRO = MAPA_TIPO_gdf[MAPA_TIPO_gdf['FILTRADO'] >= filtro]

# Guardar el resultado final
MAPA_TIPO_gdf_FILTRO.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias_Filtrado', driver='GPKG')

## Paso siete: Representación gráfica

In [ ]:
#DATOS PARA GRÁFICA

# Ruta del mapa rápido
mapa_rapido_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\CELAYA\mapa_tipologias.png"

In [ ]:
# Paso 7: Representación gráfica rápida (desde código)
import matplotlib.pyplot as plt

def plot_map_with_legend(gdf, colormap, legend_labels, legend_title, save_path):
    fig, ax = plt.subplots(1, 1)

    # Mapear cada categoría a su color correspondiente
    for category, color in colormap.items():
        MAPA_TIPO_gdf_FILTRO[MAPA_TIPO_gdf_FILTRO['TIPOLOGIA'] == category].plot(ax=ax, color=color, label=category, edgecolor='none')

    # Crear leyenda
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in colormap.values()]
    ax.legend(handles, legend_labels, title=legend_title)

    # Eliminar línea de contorno
    ax.axis('off')

    # Guardar el mapa como imagen
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# Definir el diccionario de colores
colormap = {
    'DH': '#882A1C',
    'A1': '#E33F22',
    'A2': '#E5B636',
    'A3': '#FAF36A',
    'B1': '#CAEA60',
    'B2': '#60D512',
    'B3': '#39781C',
    'B3\'': '#254D12'
}

# Definir etiquetas para la leyenda
legend_labels = list(colormap.keys())

# Título de la leyenda
legend_title = 'Tipología'

# Ruta donde se guardará el mapa
map_image_path = mapa_rapido_ruta

# Llamar a la función para generar y guardar el mapa
plot_map_with_legend(MAPA_TIPO_gdf_FILTRO, colormap, legend_labels, legend_title, map_image_path)
